In [1]:
import numpy as np
from neoradium import Carrier, PDSCH, CdlChannel, AntennaPanel, LdpcEncoder
from neoradium.utils import random

random.setSeed(123)                                        # Making the results reproducible.

carrier = Carrier(numRbs=51, spacing=30)                   # Carrier with 51 resource block, 30KHz Spacing
pdsch = PDSCH(carrier.curBwp, numLayers=2, nID=carrier.cellId)                   # PDSCH object with 2-Layers
pdsch.setDMRS(configType=2, additionalPos=1)                                     # Configuring DMRS
ldpcEncoder = LdpcEncoder(baseGraphNo=1, modulation=pdsch.modems[0].modulation,  # LDPC encoder
                          txLayers=pdsch.numLayers, targetRate=490/1024)
channel = CdlChannel('C', delaySpread=300, carrierFreq=4e9, dopplerShift=5,      # CDL-C, 300 ns, 4 GHz, 5 Hz
                     txAntenna = AntennaPanel([2,4], polarization="x"),          # 16 TX antenna
                     rxAntenna = AntennaPanel([1,2], polarization="x"))          # 4 RX antenna

grid = pdsch.getGrid()                                     # Create a resource grid (pre-populated with DMRS) 
txBlockSize = pdsch.getTxBlockSize(ldpcEncoder.targetRate) # Get the Transport Block Size 
txBlock = random.bits(txBlockSize[0])                      # Create random binary data     
numBits = pdsch.getBitSizes(grid)                          # Actual number of bits available in the grid
codeBlocks = ldpcEncoder.getRateMatchedCodeWords(txBlock, numBits[0]) # Segmentation, rate-matching, encoding
pdsch.populateGrid(grid, codeBlocks)                       # Populate the grid with modulated coded data

channelMatrix = channel.getChannelMatrix(carrier.curBwp)   # Get the channel matrix
precoder = pdsch.getPrecodingMatrix(channelMatrix)         # Get the precoder matrix from the PDSCH object
precodedGrid = grid.precode(precoder)                      # Perform the precoding

txWaveform = precodedGrid.ofdmModulate()                   # OFDM Modulation (txWaveform is in time domain)

maxDelay = channel.getMaxDelay()                           # Get channel delay
txWaveform = txWaveform.pad(maxDelay)                      # Pad zeros to the end of waveform

rxWaveform = channel.applyToSignal(txWaveform)             # Apply channel to the transmitted signal
noisyRxWaveform = rxWaveform.addNoise(snrDb=20, nFFT=carrier.curBwp.nFFT)     # Add noise to the waveform

# Synchronization
offset = channel.getTimingOffset()                   
syncedWaveform = noisyRxWaveform.sync(offset)

rxGrid = syncedWaveform.ofdmDemodulate(carrier.curBwp)                        # OFDM Demodulation
estChannelMatrix, _ = rxGrid.estimateChannelLS(pdsch.dmrs)                    # Channel estimation
eqGrid, llrScales = rxGrid.equalize(estChannelMatrix)                         # Equalization

pdschIndexes = pdsch.getReIndexes(grid, "PDSCH")                              # Indexes of the PDSCH data
llrs = pdsch.getLLRsFromGrid(eqGrid, pdschIndexes, llrScales)                 # Log-Likelihood Ratios

ldpcDecoder = ldpcEncoder.getDecoder()                                        # Get an LDPC decoder
rxCodedBlocks = ldpcDecoder.recoverRate(llrs[0], txBlockSize[0])              # Rate recovery
decodedBlocks = ldpcDecoder.decode(rxCodedBlocks)                             # LDPC-Decoding
decodedTxBlockWithCRC, crcMatch = ldpcDecoder.checkCrcAndMerge(decodedBlocks) # Checking CRC (Code-blocks)
txBlockCrcMatch = ldpcDecoder.checkCrc(decodedTxBlockWithCRC,'24A')           # Checking CRC (Transport-block)
decodedTxBlock = decodedTxBlockWithCRC[:-24]                                  # remove the transport-block CRC
print("The number of bit mismatches between the decoded transport block and original:",
      np.abs(decodedTxBlock-txBlock).sum())                                   # Compare bit-streams

The number of bit mismatches between the decoded transport block and original: 0
